# initial

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# Streaming events

In [2]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context,
)
import asyncio
from llama_index.llms.openai import OpenAI
from llama_index.utils.workflow import draw_all_possible_flows

In [3]:
class FirstEvent(Event):
    first_output: str


class SecondEvent(Event):
    second_output: str
    response: str


class ProgressEvent(Event):
    msg: str

In [4]:
class MyWorkflow(Workflow):
    @step
    async def step_one(self, ctx: Context, ev: StartEvent) -> FirstEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="Step one is happening"))
        return FirstEvent(first_output="First step complete.")

    @step
    async def step_two(self, ctx: Context, ev: FirstEvent) -> SecondEvent:
        llm = OpenAI(model="gpt-4o-mini")
        generator = await llm.astream_complete(
            "Please give me the first 3 paragraphs of Moby Dick, a book in the public domain."
        )
        async for response in generator:
            # Allow the workflow to stream this piece of response
            ctx.write_event_to_stream(ProgressEvent(msg=response.delta))
        return SecondEvent(
            second_output="Second step complete, full response attached",
            response=str(response),
        )

    @step
    async def step_three(self, ctx: Context, ev: SecondEvent) -> StopEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="Step three is happening"))
        return StopEvent(result="Workflow complete.")

In [6]:
w = MyWorkflow(timeout=300, verbose=True)
handler = w.run(first_input="Start the workflow.")

async for ev in handler.stream_events():
    if isinstance(ev, ProgressEvent):
        print(ev.msg)

final_result = await handler
print("Final result", final_result)

# draw_all_possible_flows(MyWorkflow, filename="streaming_workflow.html")


Running step step_one
Step step_one produced event FirstEvent
Running step step_two
Step one is happening

Sure
!
 Here
 are
 the
 first
 three
 paragraphs
 of
 "
M
oby
 Dick
"
 by
 Herman
 Mel
ville
:


---


Call
 me
 Ish
mael
.
 Some
 years
 ago
—
never
 mind
 how
 long
 precisely
—I
 had
 little
 or
 no
 money
 in
 my
 purse
,
 and
 nothing
 particular
 to
 interest
 me
 on
 shore
.
 So
,
 whenever
 I
 find
 myself
 growing
 grim
 about
 the
 mouth
;
 whenever
 it
 is
 a
 damp
,
 dr
izzly
 November
 in
 my
 soul
;
 whenever
 I
 find
 myself
 invol
unt
arily
 pa
using
 before
 coffin
 warehouses
,
 and
 bringing
 up
 the
 rear
 of
 every
 funeral
 I
 meet
;
 and
 especially
 whenever
 my
 hy
pos
 get
 such
 an
 upper
 hand
 of
 me
,
 that
 it
 requires
 a
 strong
 moral
 principle
 to
 prevent
 me
 from
 deliberately
 stepping
 into
 the
 street
,
 and
 method
ically
 knocking
 people
’s
 hats
 off
—
then
,
 I
 account
 it
 high
 time
 to
 get
 to
 sea
 as
 soon
 as
 I
 can
.
 This
